Завантаження та попередня обробка даних

In [13]:
import pandas as pd
import torch
from transformers import BertTokenizer, BertForSequenceClassification, AdamW, get_linear_schedule_with_warmup
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler, TensorDataset, SubsetRandomSampler
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from torch.cuda.amp import autocast, GradScaler
import numpy as np
from tqdm import tqdm

#to avoid warnings
import warnings
warnings.filterwarnings('ignore')

# Завантаження даних
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')

# Вибір потрібних колонок
comments = train_df['comment_text'].values
labels = train_df[['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']].values

# Розділення на тренувальні та валідаційні дані
train_comments, val_comments, train_labels, val_labels = train_test_split(comments, labels, test_size=0.1)

In [14]:
train_df.head()

,Unnamed: 0,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,54995,92f82e560302081c,Hi! \n\nI wanna rape you!,1,0,1,0,0,0
1,22018,3a0757419577668d,Terrorize \nI will terrorise you for as long a...,1,0,1,1,1,0
2,9074,1829ec769291fb9a,Being blocked\n\nSo that's your idea of mediat...,1,0,0,0,0,0
3,40050,6ae4580294f2d587,"Editing Wikipedia \n\nFuck You, you anal rapist",1,0,1,0,1,0
4,132307,c400904764e4138c,So the fact that the US Govt wants to put some...,1,0,0,0,0,0


In [15]:
test_df.head()

,Unnamed: 0,id,comment_text
0,0,00001cee341fdb12,Yo bitch Ja Rule is more succesful then you'll...
1,1,0000247867823ef7,#ERROR!
2,2,00013b17ad220c46,""" \n\n == Sources == \n\n * Zawe Ashton on Lap..."
3,3,00017563c3f7919a,":If you have a look back at the source, the in..."
4,4,00017695ad8997eb,I don't anonymously edit articles at all.


Попередня обробка тексту

In [16]:
# Ініціалізація токенізатора BERT
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Функція токенізації та перетворення тексту в тензори
def encode_comments(comments, tokenizer, max_length=512):
    input_ids = []
    attention_masks = []

    for comment in comments:
        encoded = tokenizer.encode_plus(
            comment,
            add_special_tokens=True,
            max_length=max_length,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt'
        )
        input_ids.append(encoded['input_ids'])
        attention_masks.append(encoded['attention_mask'])

    return torch.cat(input_ids, dim=0), torch.cat(attention_masks, dim=0)

# Токенізація тренувальних та валідаційних коментарів
train_input_ids, train_attention_masks = encode_comments(train_comments, tokenizer)
val_input_ids, val_attention_masks = encode_comments(val_comments, tokenizer)

Налаштування моделі BERT

In [17]:
# Створення тензорних датасетів
train_dataset = TensorDataset(train_input_ids, train_attention_masks, torch.tensor(train_labels, dtype=torch.float32))
val_dataset = TensorDataset(val_input_ids, val_attention_masks, torch.tensor(val_labels, dtype=torch.float32))

# Налаштування batch_size
batch_size = 16 

# Обмеження кроків на епоху 
steps_per_epoch = 1000
samples_per_epoch = batch_size * steps_per_epoch

# Обрання випадкової підмножини індексів для тренувального датасету
train_indices = np.random.choice(len(train_dataset), size=samples_per_epoch, replace=False)
train_sampler = SubsetRandomSampler(train_indices)

# Створення даталоадерів
train_dataloader = DataLoader(train_dataset, sampler=train_sampler, batch_size=batch_size)
val_dataloader = DataLoader(val_dataset, sampler=SequentialSampler(val_dataset), batch_size=batch_size)

# Ініціалізація моделі BERT
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=6)

# Використання GPU, якщо доступно
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

# Налаштування оптимізатора та планувальника
from transformers import AdamW, get_linear_schedule_with_warmup

optimizer = AdamW(model.parameters(), lr=1e-5, eps=1e-8)
epochs = 3
total_steps = steps_per_epoch * epochs
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=total_steps)

# Вибір функції втрат для багатоміткової класифікації
criterion = torch.nn.BCEWithLogitsLoss()

# Ініціалізація скейлера для змішаної точності
scaler = GradScaler()

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Тренування моделі

In [18]:
# Тренування моделі
for epoch in range(epochs):
    print(f'Epoch {epoch + 1}/{epochs}')
    
    # Тренувальна фаза
    model.train()
    total_train_loss = 0
    
    # Додаємо прогрес-бар до тренувального циклу
    for step, batch in enumerate(tqdm(train_dataloader, desc="Training")):
        b_input_ids, b_input_mask, b_labels = tuple(t.to(device) for t in batch)
        
        model.zero_grad()
        
        # Використання змішаної точності
        with autocast():
            outputs = model(b_input_ids, attention_mask=b_input_mask)
            logits = outputs.logits
            loss = criterion(logits, b_labels)
        
        # Масштабування градієнтів
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()
        scheduler.step()

        total_train_loss += loss.item()

    avg_train_loss = total_train_loss / len(train_dataloader)
    print(f'Training loss: {avg_train_loss}')

Epoch 1/3


Training: 100%|██████████| 32/32 [44:27<00:00, 83.37s/it] 


Training loss: 0.605981444939971
Epoch 2/3


Training: 100%|██████████| 32/32 [43:01<00:00, 80.66s/it]


Training loss: 0.5414233421906829
Epoch 3/3


Training:   6%|▋         | 2/32 [05:35<1:23:46, 167.55s/it]


KeyboardInterrupt: 

Валідаційна фаза

In [ ]:
model.eval()
total_val_loss = 0
predictions, true_labels = [], []

# Додаємо прогрес-бар до валідаційного циклу
for batch in tqdm(val_dataloader, desc="Validation"):
    b_input_ids, b_input_mask, b_labels = tuple(t.to(device) for t in batch)
    
    with torch.no_grad():
        with autocast():  # Використовуємо змішану точність і тут
            outputs = model(b_input_ids, attention_mask=b_input_mask)
            logits = outputs.logits
            loss = criterion(logits, b_labels)
            
    total_val_loss += loss.item()
    predictions.append(logits)
    true_labels.append(b_labels)

avg_val_loss = total_val_loss / len(val_dataloader)
print(f'Validation loss: {avg_val_loss}')

# Оцінка моделі
flat_predictions = np.concatenate([pred.cpu().numpy() for pred in predictions], axis=0)
flat_true_labels = np.concatenate([label.cpu().numpy() for label in true_labels], axis=0)
report = classification_report(flat_true_labels, flat_predictions > 0.5)
print(report)

Validation: 100%|██████████| 998/998 [01:14<00:00, 13.40it/s]

Validation loss: 0.04618379559119667
              precision    recall  f1-score   support

           0       0.85      0.79      0.82      1514
           1       0.78      0.05      0.09       139
           2       0.84      0.79      0.81       827
           3       0.00      0.00      0.00        39
           4       0.78      0.68      0.73       800
           5       0.00      0.00      0.00       126

   micro avg       0.83      0.70      0.76      3445
   macro avg       0.54      0.39      0.41      3445
weighted avg       0.79      0.70      0.73      3445
 samples avg       0.07      0.06      0.06      3445



Збереження моделі

In [ ]:
# Збереження моделі
model.save_pretrained('Saved_model')
tokenizer.save_pretrained('Saved_model')

('Saved_model\\tokenizer_config.json',
 'Saved_model\\special_tokens_map.json',
 'Saved_model\\vocab.txt',
 'Saved_model\\added_tokens.json')